In [1]:
import pandas as pd

In [8]:
selloutData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")

In [9]:
selloutData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0


### 1. 그룹단위(item) 단위로 데이터 사이즈 확인

In [179]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]

In [34]:
selloutData.groupby(groupKey)["YEARWEEK"].agg(["size"])
selloutData.groupby(groupKey)["YEARWEEK"].agg(["mean"]).head(1)

,mean
ITEM,
ITEM001,201545.0


In [181]:
groupData = selloutData.groupby(groupKey)["YEARWEEK"].agg(["size"]).reset_index()

In [182]:
groupData.rename(columns={"size":"KNOB"},inplace=True)

In [183]:
groupData.shape

(491, 5)

In [184]:
selloutData.shape

(32415, 12)

### 2.원본 데이터와 그룹핑데이터 머지

In [185]:
mergedData = pd.merge(left=selloutData,
         right=groupData,
         how="left",
         on=groupKey
        )

In [186]:
mergedData.shape

(32415, 13)

In [187]:
maxKnob = mergedData.KNOB.max()

### 3. 데이터 정제 시작(과거실적 최대치만 남김)

In [188]:
cleansedData = mergedData[mergedData.KNOB >=  maxKnob ]

In [189]:
cleansedData.shape

(7592, 13)

In [190]:
len(cleansedData.ITEM.unique())

52

In [68]:
cleansedData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
KNOB              int64
dtype: object

In [69]:
from sklearn.preprocessing import LabelEncoder
le_yn = LabelEncoder()

In [70]:
cleansedData["LE_HOLI"] = le_yn.fit_transform(cleansedData.HOLIDAY)
cleansedData["LE_PROMO"] = le_yn.fit_transform(cleansedData.PROMOTION)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
cleansedData.head(3)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,KNOB,LE_HOLI,LE_PROMO
2352,A01,PG02,P03,ITEM043,201501,2015,1,87,Y,1,Y,0.19759,146,1,1
2353,A01,PG02,P03,ITEM043,201502,2015,2,60,N,4,Y,0.19759,146,0,1
2354,A01,PG02,P03,ITEM043,201503,2015,3,51,N,4,N,0.00000,146,0,0


In [48]:
cleansedData.shape

(7592, 15)

### 1번 풀이 -> groupby 풀이

### item 별로 예측

In [71]:
predictGroup = cleansedData.groupby(groupKey)

In [72]:
list(predictGroup.groups)[51]

'ITEM376'

In [73]:
onegroup = predictGroup.get_group(list(predictGroup.groups)[51])

In [74]:
onegroup.shape

(146, 15)

In [75]:
onegroup = onegroup.reset_index(drop=True)

In [76]:
onegroup.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,KNOB,LE_HOLI,LE_PROMO
0,A01,PG05,P15,ITEM376,201501,2015,1,170,Y,1,Y,0.195745,146,1,1
1,A01,PG05,P15,ITEM376,201502,2015,2,119,N,4,Y,0.195745,146,0,1


### 2번 풀이->category 풀이

In [85]:
le_item = LabelEncoder()
le_regionid = LabelEncoder()
le_productgroup = LabelEncoder()
le_product = LabelEncoder()

In [88]:
cleansedData["LE_HOLI"] = le_yn.fit_transform(cleansedData.HOLIDAY)
cleansedData["LE_PROMO"] = le_yn.fit_transform(cleansedData.PROMOTION)
cleansedData["LE_ITEM"] = le_item.fit_transform(cleansedData.ITEM)
cleansedData["LE_REGION"] = le_item.fit_transform(cleansedData.REGIONID)
cleansedData["LE_PROD_GROUP"] = le_item.fit_transform(cleansedData.PRODUCTGROUP)
cleansedData["LE_PROD"] = le_item.fit_transform(cleansedData.PRODUCT)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [90]:
cleansedData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,KNOB,LE_HOLI,LE_PROMO,LE_ITEM,LE_REGION,LE_PROD_GROUP,LE_PROD
2352,A01,PG02,P03,ITEM043,201501,2015,1,87,Y,1,Y,0.197590,146,1,1,0,0,0,0
2353,A01,PG02,P03,ITEM043,201502,2015,2,60,N,4,Y,0.197590,146,0,1,0,0,0,0
2354,A01,PG02,P03,ITEM043,201503,2015,3,51,N,4,N,0.000000,146,0,0,0,0,0,0
2355,A01,PG02,P03,ITEM043,201504,2015,4,37,Y,2,N,0.000000,146,1,0,0,0,0,0
2356,A01,PG02,P03,ITEM043,201505,2015,5,136,N,4,Y,0.201205,146,0,1,0,0,0,0


In [79]:
le_item.inverse_transform(cleansedData.LE_ITEM)

array(['ITEM043', 'ITEM043', 'ITEM043', ..., 'ITEM376', 'ITEM376',
       'ITEM376'], dtype=object)

In [82]:
cleansedData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,KNOB,LE_HOLI,LE_PROMO,LE_ITEM
2352,A01,PG02,P03,ITEM043,201501,2015,1,87,Y,1,Y,0.197590,146,1,1,0
2353,A01,PG02,P03,ITEM043,201502,2015,2,60,N,4,Y,0.197590,146,0,1,0
2354,A01,PG02,P03,ITEM043,201503,2015,3,51,N,4,N,0.000000,146,0,0,0
2355,A01,PG02,P03,ITEM043,201504,2015,4,37,Y,2,N,0.000000,146,1,0,0
2356,A01,PG02,P03,ITEM043,201505,2015,5,136,N,4,Y,0.201205,146,0,1,0


In [93]:
corrdf = cleansedData.corr()
corrdf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,KNOB,LE_HOLI,LE_PROMO,LE_ITEM,LE_REGION,LE_PROD_GROUP,LE_PROD
YEARWEEK,1.000000e+00,9.832932e-01,2.392348e-02,0.024869,8.534880e-03,0.244636,NaN,5.108832e-03,0.094830,0.000000e+00,NaN,4.751344e-18,8.518743e-18
YEAR,9.832932e-01,1.000000e+00,-1.584529e-01,-0.002302,6.501308e-02,0.203425,NaN,-4.072250e-02,0.084591,-2.500084e-18,NaN,3.095853e-18,-4.122495e-16
WEEK,2.392348e-02,-1.584529e-01,1.000000e+00,0.147539,-3.107608e-01,0.209740,NaN,2.513620e-01,0.049804,-1.990940e-18,NaN,-1.431800e-17,-1.829053e-17
QTY,2.486926e-02,-2.302016e-03,1.475395e-01,1.000000,-2.597564e-01,0.347993,NaN,2.246354e-01,0.112769,1.439177e-01,NaN,1.510086e-01,1.747543e-01
HCLUS,8.534880e-03,6.501308e-02,-3.107608e-01,-0.259756,1.000000e+00,-0.335141,NaN,-9.768332e-01,-0.093221,-5.490514e-19,NaN,4.220885e-17,-9.760642e-18
PRO_PERCENT,2.446360e-01,2.034249e-01,2.097402e-01,0.347993,-3.351410e-01,1.000000,NaN,2.859627e-01,0.725557,1.843868e-01,NaN,2.196276e-01,1.650151e-01
KNOB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LE_HOLI,5.108832e-03,-4.072250e-02,2.513620e-01,0.224635,-9.768332e-01,0.285963,NaN,1.000000e+00,0.081870,-6.071775e-19,NaN,3.832234e-18,-3.832974e-17
LE_PROMO,9.483034e-02,8.459070e-02,4.980427e-02,0.112769,-9.322107e-02,0.725557,NaN,8.186992e-02,1.000000,1.059897e-01,NaN,1.412269e-01,7.386776e-02
LE_ITEM,0.000000e+00,-2.500084e-18,-1.990940e-18,0.143918,-5.490514e-19,0.184387,NaN,-6.071775e-19,0.105990,1.000000e+00,NaN,9.130656e-01,9.437133e-01


In [162]:
all_data = []
features = list (corrdf[ (abs(corrdf.QTY) > 0.15) & (abs(corrdf.QTY) < 1)].index)
features.append('LE_ITEM')
label = ["QTY"]
all_data = features[0:]
all_data.append(label[0])
all_data
features

['HCLUS', 'PRO_PERCENT', 'LE_HOLI', 'LE_PROD_GROUP', 'LE_PROD', 'LE_ITEM']

In [163]:
featuresData = cleansedData[all_data]
featuresData = featuresData.reset_index(drop=True)

In [164]:
from sklearn.model_selection import train_test_split

In [165]:
trainingData_features,testData_features,trainingData_label,testData_label = \
train_test_split(featuresData[features],featuresData[label], test_size=0.2, random_state=1)

In [191]:
trainingData_features

,HCLUS,PRO_PERCENT,LE_HOLI,LE_PROD_GROUP,LE_PROD,LE_ITEM
2722,4,0.166537,0,2,3,18
4406,1,0.304350,1,2,3,30
6997,4,0.000000,0,3,6,47
6159,1,0.318242,1,3,5,42
6062,4,0.409013,0,3,5,41
1615,4,0.000000,0,1,2,11
6146,4,0.262180,0,3,5,42
181,1,0.249246,1,0,0,1
5947,2,0.235609,1,2,4,40
5787,2,0.291705,1,2,4,39


### 3. 모델 생성

In [166]:
inputDim = trainingData_features.loc[0,:].shape
inputDim

(6,)

In [167]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import neighbors

In [168]:
model = Sequential()
model.add(Dense(units=8, activation="relu",input_shape=inputDim))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=1, activation="relu"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 56        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 137
Trainable params: 137
Non-trainable params: 0
_________________________________________________________________


### 4. 모델 컴파일

In [169]:
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics=['mean_squared_error','mean_absolute_error'])

### 5. 모델 훈련(callback 정의)

In [170]:
#callback List를 만들기 위한 선언 및 변수 설정
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from datetime import datetime
import os

log_folder = datetime.now().strftime("%Y%m%d_%H%M%S")



log_dirs = os.path.join("logs",log_folder)

In [171]:
callback_list = [
    TensorBoard(log_dir = log_dirs),
    EarlyStopping(monitor="val_loss", patience=10), 
    ModelCheckpoint(filepath="./model_r_weights.h5", monitor="val_loss", verbose=1, save_best_only=True)   
]

In [172]:
model.fit(x=trainingData_features,
          y=trainingData_label,
          batch_size=32,
          epochs=2000,
          validation_split=0.2,
          callbacks=callback_list)

W1016 15:52:03.373573  8284 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 4858 samples, validate on 1215 samples
Epoch 1/2000
4384/4858 [==========================>...] - ETA: 0s - loss: 146666.8189 - mean_squared_error: 146666.8125 - mean_absolute_error: 166.6271
Epoch 00001: val_loss improved from inf to 132999.97621, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 1s 141us/sample - loss: 146960.4601 - mean_squared_error: 146960.4531 - mean_absolute_error: 167.0422 - val_loss: 132999.9762 - val_mean_squared_error: 132999.9688 - val_mean_absolute_error: 154.1216
Epoch 2/2000
4416/4858 [==========================>...] - ETA: 0s - loss: 132352.2782 - mean_squared_error: 132352.3281 - mean_absolute_error: 153.8720
Epoch 00002: val_loss improved from 132999.97621 to 115273.30388, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 58us/sample - loss: 135553.1392 - mean_squared_error: 135553.1875 - mean_absolute_error: 155.7233 - val_loss: 115273.3039 - val_mean_squared_error: 115273.320

Epoch 18/2000
4416/4858 [==========================>...] - ETA: 0s - loss: 105613.1709 - mean_squared_error: 105613.1719 - mean_absolute_error: 174.9903
Epoch 00018: val_loss improved from 96144.64316 to 95478.19129, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 57us/sample - loss: 106281.5182 - mean_squared_error: 106281.5234 - mean_absolute_error: 175.1986 - val_loss: 95478.1913 - val_mean_squared_error: 95478.1953 - val_mean_absolute_error: 168.6806
Epoch 19/2000
3904/4858 [=======================>......] - ETA: 0s - loss: 106800.2583 - mean_squared_error: 106800.2812 - mean_absolute_error: 173.7615
Epoch 00019: val_loss improved from 95478.19129 to 94951.01161, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 64us/sample - loss: 105775.7925 - mean_squared_error: 105775.8047 - mean_absolute_error: 174.0852 - val_loss: 94951.0116 - val_mean_squared_error: 94951.0000 - val_mean_absolute_error: 172.4890
Epoch 20

Epoch 35/2000
4608/4858 [===========================>..] - ETA: 0s - loss: 101375.2562 - mean_squared_error: 101375.2422 - mean_absolute_error: 172.7732
Epoch 00035: val_loss improved from 91390.62014 to 91216.22498, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 89us/sample - loss: 102455.7858 - mean_squared_error: 102455.7734 - mean_absolute_error: 173.7456 - val_loss: 91216.2250 - val_mean_squared_error: 91216.2109 - val_mean_absolute_error: 168.1683
Epoch 36/2000
4256/4858 [=========================>....] - ETA: 0s - loss: 107283.0384 - mean_squared_error: 107283.0078 - mean_absolute_error: 178.0132
Epoch 00036: val_loss improved from 91216.22498 to 91082.37449, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 61us/sample - loss: 102300.3046 - mean_squared_error: 102300.2812 - mean_absolute_error: 174.7452 - val_loss: 91082.3745 - val_mean_squared_error: 91082.3750 - val_mean_absolute_error: 165.7242
Epoch 37

Epoch 52/2000
4736/4858 [============================>.] - ETA: 0s - loss: 98064.1883 - mean_squared_error: 98064.1484 - mean_absolute_error: 168.9024  
Epoch 00052: val_loss improved from 89329.85173 to 89159.90775, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 56us/sample - loss: 99922.7886 - mean_squared_error: 99922.7500 - mean_absolute_error: 169.6481 - val_loss: 89159.9078 - val_mean_squared_error: 89159.9062 - val_mean_absolute_error: 166.8476
Epoch 53/2000
4032/4858 [=======================>......] - ETA: 0s - loss: 107588.3470 - mean_squared_error: 107588.3281 - mean_absolute_error: 176.7632
Epoch 00053: val_loss improved from 89159.90775 to 89011.42614, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 69us/sample - loss: 99627.3330 - mean_squared_error: 99627.3203 - mean_absolute_error: 172.9415 - val_loss: 89011.4261 - val_mean_squared_error: 89011.4219 - val_mean_absolute_error: 161.9789
Epoch 54/200

...

4512/4858 [==========================>...] - ETA: 0s - loss: 83756.0201 - mean_squared_error: 83756.0391 - mean_absolute_error: 147.8941
Epoch 00261: val_loss did not improve from 72744.85910
4858/4858 [==============================] - 0s 42us/sample - loss: 79802.3394 - mean_squared_error: 79802.3516 - mean_absolute_error: 145.0318 - val_loss: 72823.1911 - val_mean_squared_error: 72823.2031 - val_mean_absolute_error: 136.9973
Epoch 262/2000
4352/4858 [=========================>....] - ETA: 0s - loss: 75295.0246 - mean_squared_error: 75295.0156 - mean_absolute_error: 141.5455
Epoch 00262: val_loss did not improve from 72744.85910
4858/4858 [==============================] - 0s 55us/sample - loss: 79867.4866 - mean_squared_error: 79867.4844 - mean_absolute_error: 143.8691 - val_loss: 72933.5498 - val_mean_squared_error: 72933.5469 - val_mean_absolute_error: 141.1456
Epoch 263/2000
4128/4858 [========================>.....] - ETA: 0s - loss: 77715.7789 - mean_squared_error: 77715.7891 -

Epoch 279/2000
4064/4858 [========================>.....] - ETA: 0s - loss: 79467.1153 - mean_squared_error: 79467.1172 - mean_absolute_error: 142.8307
Epoch 00279: val_loss did not improve from 72438.69470
4858/4858 [==============================] - 0s 48us/sample - loss: 79275.0584 - mean_squared_error: 79275.0547 - mean_absolute_error: 143.5811 - val_loss: 73158.3193 - val_mean_squared_error: 73158.3125 - val_mean_absolute_error: 143.7265
Epoch 280/2000
4256/4858 [=========================>....] - ETA: 0s - loss: 80369.1403 - mean_squared_error: 80369.1641 - mean_absolute_error: 143.8126
Epoch 00280: val_loss improved from 72438.69470 to 72396.43558, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 70us/sample - loss: 79336.7099 - mean_squared_error: 79336.7422 - mean_absolute_error: 144.2766 - val_loss: 72396.4356 - val_mean_squared_error: 72396.4375 - val_mean_absolute_error: 137.4846
Epoch 281/2000
4544/4858 [===========================>..] - 

Epoch 297/2000
4480/4858 [==========================>...] - ETA: 0s - loss: 80011.1250 - mean_squared_error: 80011.1250 - mean_absolute_error: 144.7979
Epoch 00297: val_loss did not improve from 72202.46209
4858/4858 [==============================] - 0s 73us/sample - loss: 79026.4768 - mean_squared_error: 79026.4922 - mean_absolute_error: 143.6576 - val_loss: 72207.2594 - val_mean_squared_error: 72207.2578 - val_mean_absolute_error: 137.6226
Epoch 298/2000
3808/4858 [======================>.......] - ETA: 0s - loss: 77065.2974 - mean_squared_error: 77065.2969 - mean_absolute_error: 143.6055
Epoch 00298: val_loss improved from 72202.46209 to 72118.96077, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 54us/sample - loss: 78807.6421 - mean_squared_error: 78807.6328 - mean_absolute_error: 143.2914 - val_loss: 72118.9608 - val_mean_squared_error: 72118.9609 - val_mean_absolute_error: 138.1645
Epoch 299/2000
4416/4858 [==========================>...] - 

Epoch 315/2000
4384/4858 [==========================>...] - ETA: 0s - loss: 75280.7061 - mean_squared_error: 75280.7031 - mean_absolute_error: 142.3952
Epoch 00315: val_loss improved from 71899.88914 to 71834.87572, saving model to ./model_r_weights.h5
4858/4858 [==============================] - 0s 63us/sample - loss: 78248.1905 - mean_squared_error: 78248.1797 - mean_absolute_error: 143.0312 - val_loss: 71834.8757 - val_mean_squared_error: 71834.8750 - val_mean_absolute_error: 135.5228
Epoch 316/2000
4672/4858 [===========================>..] - ETA: 0s - loss: 78125.0654 - mean_squared_error: 78125.0703 - mean_absolute_error: 141.9472
Epoch 00316: val_loss did not improve from 71834.87572
4858/4858 [==============================] - 0s 42us/sample - loss: 78408.8208 - mean_squared_error: 78408.8203 - mean_absolute_error: 142.3495 - val_loss: 72017.5933 - val_mean_squared_error: 72017.6016 - val_mean_absolute_error: 139.7223
Epoch 317/2000
4448/4858 [==========================>...] - 

Epoch 333/2000
4672/4858 [===========================>..] - ETA: 0s - loss: 78983.0064 - mean_squared_error: 78982.9922 - mean_absolute_error: 142.2566
Epoch 00333: val_loss did not improve from 71488.62831
4858/4858 [==============================] - 0s 81us/sample - loss: 77828.9951 - mean_squared_error: 77828.9766 - mean_absolute_error: 141.6152 - val_loss: 71597.0320 - val_mean_squared_error: 71597.0234 - val_mean_absolute_error: 134.5893
Epoch 334/2000
4416/4858 [==========================>...] - ETA: 0s - loss: 80375.9496 - mean_squared_error: 80375.9453 - mean_absolute_error: 143.2997
Epoch 00334: val_loss did not improve from 71488.62831
4858/4858 [==============================] - 0s 73us/sample - loss: 77910.0418 - mean_squared_error: 77910.0469 - mean_absolute_error: 141.4719 - val_loss: 71495.4056 - val_mean_squared_error: 71495.3984 - val_mean_absolute_error: 134.4475
Epoch 335/2000
4288/4858 [=========================>....] - ETA: 0s - loss: 70904.8582 - mean_squared_erro

### 6. 모델 추론(validation)

In [173]:
model.evaluate(x=testData_features, y=testData_label)


W1016 15:54:04.194237  8284 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


1519/1519 [==============================] - 0s 33us/sample - loss: 60207.4531 - mean_squared_error: 60207.4531 - mean_absolute_error: 134.8324


[60207.45310185566, 60207.453, 134.83241]

In [174]:
lose, mse, mae = model.evaluate(x=testData_features,
              y=testData_label)

W1016 15:54:04.267644  8284 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


1519/1519 [==============================] - 0s 41us/sample - loss: 60207.4531 - mean_squared_error: 60207.4531 - mean_absolute_error: 134.8324


In [177]:
# LE_HOLI', 'LE_PROD_GROUP', 'LE_PROD'
HCLUS=3
PRO_PERCENT=0.6
LE_HOLI=1
LE_PROD_GROUP=1
LE_PROD=3
LE_ITEM=20
sampledata = pd.DataFrame([[HCLUS,PRO_PERCENT,LE_HOLI,LE_PROD_GROUP,LE_PROD,LE_ITEM]])
sampledata

,0,1,2,3,4,5
0,3,0.6,1,1,3,20


In [178]:
model.predict(sampledata)

W1016 15:55:03.560076  8284 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


array([[683.62067]], dtype=float32)